In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing
import random # later to achiev a certain task
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
dataset = pd.read_csv('Arabic_Dataset.txt', on_bad_lines='skip')

In [6]:
dataset.iloc[0]['poet_name']

'لقيط بن يعمر الإيادي'

In [8]:
dataset.drop('Unnamed: 6', axis=1)

,poet_name,poet_era,poem_tags,poem_title,poem_text,poem_count
0,لقيط بن يعمر الإيادي,العصر الجاهلي,"قصائد هجاء, عموديه, بحر الوافر, قافية الدال (د)",سلام في الصحيفة من لقيط,سَلامٌ في الصَحيفَةِ مِن لَقيطٍ\r\nإِلى مَن بِ...,عدد الابيات : 4
1,لقيط بن يعمر الإيادي,العصر الجاهلي,"قصائد حزينه, عموديه, بحر البسيط, قافية الألف ...",يا دار عمرة من محتلها الجرعا,يا دارَ عَمرَةَ مِن مُحتَلِّها الجَرَعا\r\nهاج...,عدد الابيات : 60
2,لقيط بن يعمر الإيادي,العصر الجاهلي,"قصائد قصيره, عموديه, بحر الرجز, قافية الألف (ا)",وخاننا خوان في ارتباعنا,وَخانَنا خَوّانٌ في اِرتِباعِنا\r\nفَاِنفَدَّ ...,عدد الابيات : 1
3,زبان بن سيار الفزاري,العصر الجاهلي,"قصائد قصيره, عموديه, بحر الطويل, قافية الألف ...",تنح إليكم يا ابن كوز فإننا,تَنَحَّ إِلَيكُم يا اِبنَ كوزٍ فَإِنَّنا\r\nوَ...,عدد الابيات : 1
4,زبان بن سيار الفزاري,العصر الجاهلي,"قصائد قصيره, عموديه, بحر الطويل, قافية الباء ...",تطارحه الأنساب حتى رددنه,تُطارِحُهُ الأَنسابُ حَتّى رَدَدنَهُ\r\nإِلى ن...,عدد الابيات : 1
...,...,...,...,...,...,...
80998,أبو شراعة,العصر العباسي,"قصائد حزينه, عموديه, بحر الخفيف, قافية اللام ...",تلوم جودي لبرمة الطفشيل,تَلومُ جودي لِبُرمَةِ الطَفشيلِ\r\nوَاِستَهِلّ...,عدد الابيات : 9
80999,أبو شراعة,العصر العباسي,"قصائد رومنسيه, عموديه, بحر البسيط, قافية اللا...",وردت دار سعيد وهي خالية,وَرَدتُ دارَ سَعيدٍ وَهيَ خالِيَةٌ\r\nوَكانَ أ...,عدد الابيات : 4
81000,أبو شراعة,العصر العباسي,"قصائد قصيره, عموديه, بحر الطويل, قافية الياء ...",ألا لا أبالي في العلى ما أصابني,أَلا لا أُبالي في العُلى ما أَصابَني\r\nوَإِن ...,عدد الابيات : 3
81001,أبو شراعة,العصر العباسي,"قصائد عتاب, عموديه, بحر الوافر, قافية الياء (ي)",بني سوار إن رثت ثيابي,بَني سَوّارَ إِن رَثَت ثِيابي\r\nوَكَلَّ عَنِ ...,عدد الابيات : 9


In [10]:
dataset.fillna('', inplace = True)

In [12]:
#libraries:
import pandas as pd
from math import log10


# We will base it on tf-idf on the poem_text
#so the similarity would be weighted on the rarity of words used by the user input
# we'll work with dictionnaries cz it is more efficient

#we'll create a mapping from the word to the word count in each poem and then in another function we'll compute the necessary computations
# we're doing it across ALL poems so that the tf-idf would reflect the uniquness of the charachters




def map_wrd_to_wordcount_and_poemname(dataset, poem_content_col_name, poem_name_col):
# this will give us a dictionnary with keys -> charachters/words and value -> the list: [the nb of occurence across all poems, name of poems it occured in
    nb_of_rows = len(dataset)
    dict_map = {}
    for row_i in range(nb_of_rows):
        all_charachters_in_poem_of_index_i = dataset.iloc[row_i][poem_content_col_name].split()
        #we will append it to the dicttionnary while keeping track of the names of the poems that we encountered
        poem_name = dataset.iloc[row_i][poem_name_col]
        for char in all_charachters_in_poem_of_index_i:
            #2 cases are present either the char is already in our dictionnary or is not
            if char in dict_map: 
                #2 cases are present so we have the charachter but either the specific poem is not discovered yet so we need to add to the dictionnary a new key with value 1
                if poem_name not in dict_map[char]:
                    dict_map[char][poem_name] = 1 
                #or the poem_name is already discovered and it is not the first time this charachter is mentioned for this specific poem
                else:
                    dict_map[char][poem_name] +=1
            if char not in dict_map:
                #each poem is gonna have its own term frequency 
                dict_map[char] = {poem_name:1}
    return dict_map
    #we could convert it to a dataframe so that we could better visualize it

#Remember: dict_map is gonna be the charachter mapped to the poem_name which is mapped to the word frequency
#e.g. D1 = "I love cookies", D2 = "I love football" 
#the desired output of the program is gonna be:
#{"I": {D1:1, D2:1}, "love": {D1:1, D2:1}, "cookies": {D1:1}, "football": {D2:1}}

#important notice we should NOT forget to fillnas with zeros when we transform to a dataframe
#or we take a condition later on while calculating the tfidf stating that if the document is not mentioned it should assign 0 to the mapping

#note we could transform this to a pd library using:
# mapping = pd.DataFrame.from_dict(dict_map, orient="index")

def map_wrd_to_poemname_and_tf(dataset, poem_content_col_name , poem_name_col):
# we'll do it on the contents of the poem
    dict_frequency = map_wrd_to_wordcount_and_poemname(dataset, poem_content_col_name, poem_name_col)

    nb_of_rows = len(dataset)
    #we will loop over all poems and get their length and modify the values in dict_freq
    for row in range(nb_of_rows):
        poem_content = dataset.iloc[row][poem_content_col_name].split()
        poem_name = dataset.iloc[row][poem_name_col]
        nb_words_in_poem = len(poem_content)
        #updating the dict_frequency for each value for the corresponding poem
        for char in dict_frequency:
            if poem_name in dict_frequency[char]:
                dict_frequency[char][poem_name] = dict_frequency[char][poem_name]/nb_words_in_poem
    
    #now dict_frequency is the dictionnary of frequency/total nb of words thus
    dict_tf = dict_frequency
    return dict_tf


#so now we are searching the inverse- DOCUMENT - frequency thus we can't use map_poem_to_wordcount 
# cz this counts the frequency as a whole

#document frequency:
def map_wrd_to_document_frequency(dataset, poem_content_col_name):
    dict_map = {}

    total_rows = len(dataset[poem_content_col_name]) 
    useless_rows =  sum(dataset[poem_content_col_name].isnull()) 
    nb_of_rows = total_rows - useless_rows

    for row_i in range(nb_of_rows):
        all_charachters_in_poem_of_index_i = dataset.iloc[row_i][poem_content_col_name].split()
#we won't add it the poem name because we already we added the while computing the term-frequenc mapping
#i.e. we got all the poems where a certain charachter was mentioned in the process of computing the dict_map of the tf
        for char in all_charachters_in_poem_of_index_i:
            if char not in dict_map:
                dict_map[char] = 1

    return dict_map


# the formula for idf log(1+N/1+dict_map[key])
# we're adding one to prevent division by zero and to prevent log(0)

def map_wrd_to_idf(dataset, poem_content_col_name):
# we'll do it on the contents of the poem

    dict_map_df = map_wrd_to_document_frequency(dataset, poem_content_col_name)

#it is inevitable that there are null values thus we need to 
 
    nb_of_rows = len(dataset)
    nb_of_null_rows = sum(dataset[poem_content_col_name].isnull())
    nb_non_null_poems =  nb_of_rows - nb_of_null_rows

#now we're gonna compute the idf and return the dictinary relating the charachters to their corresponding idf value
    for i in dict_map_df:
        dict_map_df[i] =  log10((1+nb_non_null_poems)/ (dict_map_df[i]+1))
# now we got the characters mapped to the idf
    return dict_map_df
#It is time for the TF-IDF mapping!:

def map_wrd_to_tfidf(dataset, poem_content_col_name, poem_name_col):
    map_wrd_to_freq_and_poems = map_wrd_to_poemname_and_tf(dataset, poem_content_col_name, poem_name_col)

#############################################
#Useless
    # map_wrd_to_poem={}
    # map_wrd_tf = {}
    # for char in map_wrd_to_freq_and_poems:
    #     map_wrd_to_poem[char] = map_wrd_to_freq_and_poems[char][1] # this will map words to the poems it was mentioned in
##############################################

    
#let's get the idf:

    map_wrd_idf = map_wrd_to_idf(dataset, poem_content_col_name)

#now we're gonna perform tf*idf by modifying the tf dict_map

    for char in map_wrd_to_freq_and_poems:
        for poem in map_wrd_to_freq_and_poems[char]:
            map_wrd_to_freq_and_poems[char][poem] = map_wrd_to_freq_and_poems[char][poem] * map_wrd_idf[char]
    
    #now the tf dict_mapping is transformed to the tf-idf mapping of each word to the corresponding poem and tf-idf score
    tf_idf_mapping = map_wrd_to_freq_and_poems
    return tf_idf_mapping

In [14]:
sam = dataset.drop('Unnamed: 6', axis=1).sample(n=1000, random_state=2)

In [16]:
sam.dropna()

,poet_name,poet_era,poem_tags,poem_title,poem_text,poem_count
6662,الفرزدق,العصر الاموي,"قصائد مدح, عموديه, بحر الكامل, قافية اللام (ل)",كانت منادمة الملوك وتاجهم,كانَت مُنادَمَةُ المُلوكِ وَتاجُهُم\r\nلِمُجاش...,عدد الابيات : 16
2822,حسان بن ثابت,المخضرمون,"قصائد قصيره, عموديه, بحر الطويل, قافية العين ...",لو شهدتني من معد عصابة,لَو شَهِدَتني مِن مَعَدٍّ عِصابَةٌ\r\nسِوى ناك...,عدد الابيات : 2
80417,خالد الكاتب,العصر العباسي,"قصائد حزينه, عموديه, بحر الكامل, قافية الياء ...",لما وقفت بدأت بالهجر,لمَّا وقَفت بدأت بالهجرِ\r\nورَمَيتني من حيثُ ...,عدد الابيات : 2
27925,شهاب الدين الخفاجي,العصر العثماني,"قصائد رثاء, عموديه, بحر أحذ الكامل, قافية الي...",من تحت همزة صدغه ألف,مِن تحت هَمْزةِ صُدْغِه ألِفٌ\r\nمِن عارِضَيْه...,عدد الابيات : 2
34160,ابن فركون,العصر الأندلسي,"قصائد مدح, عموديه, بحر البسيط, قافية الباء (ب)",بشرى بوعد لنصر الدين مرقوب,بُشْرى بوعْدٍ لنصْرِ الدين مرقوبِ\r\nأتى به مُ...,عدد الابيات : 44
...,...,...,...,...,...,...
749,الأسود النهشلي,العصر الجاهلي,بحر الكامل,خالي ابُن فارس ذي الوقوف مُطلق,خالي ابنُ فارِس ذي الوُقوف مُطلقٌ\r\nوأبي أبو ...,عدد الابيات : 2
9062,قيس بن الملوح,العصر الاموي,"قصائد حزينه, عموديه, بحر الطويل, قافية الراء ...",ألا يا حمام الأيك أجريت أدمعي,أَلا يا حَمامَ الأَيكِ أَجرَيتَ أَدمُعي\r\nوَق...,عدد الابيات : 7
41947,لسان الدين بن الخطيب,العصر المملوكي,"قصائد مدح, عموديه, بحر الكامل, قافية الراء (ر)",أنا روضة وجه الأمير محمد,أنا روْضَةٌ وجْهُ الأميرِ محمّدٍ\r\nشمْسي ودون...,عدد الابيات : 2
32927,أبو الحسن الششتري,العصر الأندلسي,"قصائد رثاء, عموديه, بحر المنسرح, قافية الميم ...",للفقر أهل فكن لهم تبعا,للِفقرِ أهلٌ فَكُن لهمْ تبعَا\r\nواعملْ على حب...,عدد الابيات : 8


In [18]:
dict_map = map_wrd_to_tfidf(sam, 'poem_text', 'poem_title')
mapping = pd.DataFrame.from_dict(dict_map, orient="index")
mapping.fillna(0)

,كانت منادمة الملوك وتاجهم,وبالمسجد الأقصى الإمام الذي اهتدى,ما على حاديهم لو كان عاجا,ليت عين الرشيد كانت تراكا,وأنبئت ذا الضرع ابن جدعان سبني,سأبكي حصينا ما تغنى حمائم,لا يخدعنك جمال صوره,المجد بحر أنتم وراده,مصابي جليل والعزاء جميل,كانت ظلوم إذا عاتبتها اعتذرت,...,فمن يهدي أخا لذئاب لو,أبدر الدين لا تخشى كسوفا,مشرف الهادي له غسن,ألمت خناس وإلمامها,ومقلة كحميا الكأس مسكرة,فظل بصحراء الأميشط بطنه,أجالت حصاهن الذواري وحيضت,وقصدك مشكور وعهدك ثابت,ولادته أرخ سروري وأمه,وكما اللواط سجية الكتاب
كانَت,0.018746,0.017529,0.014914,0.064272,0.036478,0.093083,0.04029,0.009308,0.010885,0.13497,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
مُنادَمَةُ,0.018746,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
المُلوكِ,0.018746,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
وَتاجُهُم,0.018746,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
لِمُجاشِعٍ,0.018746,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
الكُتّابِ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.337426
فَكَذا,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.337426
الحُلاقُ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.337426
سَجِيَّهُ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.337426
